# Домашнее задание

In [12]:
import pandas as pd
from functools import reduce

## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [3]:
log  = pd.read_csv('доп_материалы_к_пандас_1_и_2/visit_log.csv', delimiter=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [4]:
log[log['traffic_source'].isin(['paid', 'email'])].tail()

,timestamp,visit_id,url,region,user_id,traffic_source
18927,1550094245,1e0d086733,https://host.ru/c2382eb3d6afc8d0f3,Ukraine,10c296a6a4,paid
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid
18937,1550094323,b3b634f824,https://host.ru/39fa884393666d45fc,Russia,7d27a58fb8,email


In [5]:
def source_type(row):
    if row.traffic_source in ['yandex', 'google']:
        row['source_type'] = 'organic'
    elif row.traffic_source in ['paid', 'email']:
        if row.region == 'Russia':
            row['source_type'] = 'ad'
        else:
            row['source_type'] = 'other'
    else:
        row['source_type'] = row['traffic_source']
    return row

In [6]:
log.apply(source_type, axis=1).head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

- Прочитайте содержимое файла в датафрейм
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [7]:
urls = pd.read_csv('доп_материалы_к_пандас_1_и_2/URLs.txt')
urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [8]:
pattern = r'\/\d{8}\-'
urls[urls["url"].str.contains(pattern)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


## Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [9]:
ratings = pd.read_csv('доп_материалы_к_пандас_1_и_2/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [15]:
ratings_group = ratings.groupby('userId').agg({'movieId': 'count', 'timestamp': ['min', 'max']})
ratings_group = ratings_group[ratings_group['movieId']['count'] > 100]
ratings_group['user_life'] = ratings_group['timestamp']['max'] - ratings_group['timestamp']['min']
average_user_life = ratings_group['user_life'].mean()/3600/24
average_user_life

463.89476214829165

## Задание 4

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

In [5]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [7]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [8]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [26]:
# merge three dataframes zd, air, auto using reduce
# reduce is a method from functools module that performs a function on an iteratable
revevues_merged = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, how='outer', on='client_id'), [rzd, air, auto]).fillna(0)
revevues_merged

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,0.0,57483.0
3,114,5774.0,0.0,83.0
4,115,981.0,81.0,912.0
5,116,0.0,4.0,4834.0
6,117,0.0,13.0,98.0
7,118,0.0,173.0,0.0


In [23]:
revevues_with_address = revevues_merged.merge(client_base, how='outer')
revevues_with_address

,client_id,rzd_revenue,air_revenue,auto_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,0.0,57483.0,Левобережная 1а
3,114,5774.0,0.0,83.0,Мира 14
4,115,981.0,81.0,912.0,ЗЖБИиДК 1
5,116,0.0,4.0,4834.0,Строителей 18
6,117,0.0,13.0,98.0,Панфиловская 33
7,118,0.0,173.0,0.0,Мастеркова 4
